In [1]:
import os
import sys

"""Linux"""
os.chdir("/home/optimusprime/Desktop/peeterson/github/DeepAR_demand_prediction/Rental_Fleet_Demand_Forecast")
sys.path.append(os.path.abspath(os.path.join("/home/optimusprime/Desktop/peeterson/github/DeepAR_demand_prediction/Rental_Fleet_Demand_Forecast")))

os.chdir("/home/optimusprime/Desktop/peeterson/github/DeepAR_demand_prediction/Rental_Fleet_Demand_Forecast/libs/linux-gpu")


#from ctypes import FormatError
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import os,sys

import torch
torch.use_deterministic_algorithms(True)
import pytorch_lightning as pl
from pytorch_forecasting.data.encoders import TorchNormalizer
from pytorch_forecasting.metrics import SMAPE, RMSE
from torchmetrics import R2Score, SymmetricMeanAbsolutePercentageError, MeanSquaredError

import matplotlib.pyplot as plt
import pandas as pd
from pytorch_forecasting.data import TimeSeriesDataSet
from pytorch_forecasting.data.encoders import NaNLabelEncoder
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
import torch
from pytorch_forecasting.data.encoders import TorchNormalizer
import os,sys
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import acf,pacf
from scipy.signal import find_peaks
import operator
import statsmodels.api as sm
from itertools import combinations
import pickle
from pytorch_forecasting import Baseline
import random
from pytorch_forecasting import DeepAR,NegativeBinomialDistributionLoss
from itertools import product
from sklearn.metrics import mean_absolute_error, mean_squared_error
import optuna
from optuna.trial import TrialState
import plotly
#from deepar_RegionWise_LinuxGpu_prediction_dev import train_and_forecast
from sklearn.metrics import classification_report

"""
Set Random seed
"""

random.seed(0)
torch.manual_seed(0)
np.random.seed(0)
## additional seeding to ensure reproduciblility.
pl.seed_everything(0)

2024-04-02 18:12:43.455168: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-02 18:12:43.456317: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-02 18:12:43.475257: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 18:12:43.475275: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 18:12:43.475756: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

0

Now for the round 0, we reduce dataset even more from 20% to 5%. num-epochs to a first value of 50. number of trials to a fixed number of 30.

In [2]:
dataset_size = 0.05
num_epochs = 30
num_trials = 30
logs_directory = "/home/optimusprime/Desktop/Timothy_EXP_4/Round_0_logs"

In [3]:
"""
Import pre-processed Data (tsfresh features)

response and target are the same thing
"""

### Your code
os.chdir('/home/optimusprime/Desktop/Data_Timothy')
############ Basically df = raw_feat_df <-------------------------
df = pd.read_parquet('EXP_4_combined_dd_ss_all_covariates_encoded_further_preprocessed.parquet', engine='pyarrow')
df

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,wind_speed,rain_1h,temp,clouds_all,snow_3h,rain_3h,humidity,snow_1h,weather_description,weather_main
0,167,2021-01-07 23:00:00,4,0.0,10.0,8.0,1.0,1.0,5.0,3.0,...,7.70,0.0,276.41,0,0.0,0.0,53,0.0,16,0
1,168,2021-01-08 00:00:00,4,1.0,4.0,13.0,0.0,2.0,0.0,1.0,...,8.23,0.0,275.35,0,0.0,0.0,56,0.0,16,0
2,169,2021-01-08 01:00:00,4,2.0,1.0,9.0,0.0,0.0,1.0,2.0,...,6.70,0.0,274.53,0,0.0,0.0,61,0.0,16,0
3,170,2021-01-08 02:00:00,4,0.0,1.0,5.0,0.0,1.0,2.0,3.0,...,6.20,0.0,274.13,0,0.0,0.0,63,0.0,16,0
4,171,2021-01-08 03:00:00,4,0.0,3.0,2.0,1.0,1.0,0.0,0.0,...,4.63,0.0,273.48,0,0.0,0.0,64,0.0,16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631191,24808,2023-10-31 16:00:00,263,126.0,176.0,136.0,23.0,226.0,142.0,113.0,...,4.63,0.0,282.29,20,0.0,0.0,50,0.0,2,1
1631192,24809,2023-10-31 17:00:00,263,112.0,153.0,144.0,9.0,185.0,126.0,82.0,...,4.12,0.0,283.77,20,0.0,0.0,47,0.0,2,1
1631193,24810,2023-10-31 18:00:00,263,99.0,135.0,94.0,7.0,136.0,112.0,77.0,...,3.60,0.0,283.52,75,0.0,0.0,50,0.0,0,1
1631194,24811,2023-10-31 19:00:00,263,60.0,119.0,62.0,4.0,85.0,99.0,30.0,...,4.12,0.0,283.69,100,0.0,0.0,48,0.0,14,1


In [4]:
working_directory = '/home/optimusprime/Desktop/Timothy_EXP_4'
os.chdir(working_directory)

In [5]:
unique_values_group_no_ts_fresh = list(df['group'].unique())
print(len(unique_values_group_no_ts_fresh), "\n")
print(unique_values_group_no_ts_fresh)

66 

[4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 107, 113, 114, 116, 120, 125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153, 158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]


In [6]:
zones_time_idx_no_ts_fresh = {}
for i in unique_values_group_no_ts_fresh:
    df1 = df[df['group'] == i]
    time_idx_list = df1['time_idx'].tolist()
    first_idx = time_idx_list[0]
    last_idx = time_idx_list[-1]
    zones_time_idx_no_ts_fresh[i] = [first_idx, last_idx]
    print("Group ", i, " has ", df1.shape[0], " number of rows in df_ts_fresh \n")
    print("First idx: ", first_idx, "; Last idx: ", last_idx, "\n")

Group  4  has  24646  number of rows in df_ts_fresh 

First idx:  167 ; Last idx:  24812 

Group  12  has  24789  number of rows in df_ts_fresh 

First idx:  24 ; Last idx:  24812 

Group  13  has  24790  number of rows in df_ts_fresh 

First idx:  23 ; Last idx:  24812 

Group  24  has  24645  number of rows in df_ts_fresh 

First idx:  168 ; Last idx:  24812 

Group  41  has  24645  number of rows in df_ts_fresh 

First idx:  168 ; Last idx:  24812 

Group  42  has  24645  number of rows in df_ts_fresh 

First idx:  168 ; Last idx:  24812 

Group  43  has  24788  number of rows in df_ts_fresh 

First idx:  25 ; Last idx:  24812 

Group  45  has  24646  number of rows in df_ts_fresh 

First idx:  167 ; Last idx:  24812 

Group  48  has  24788  number of rows in df_ts_fresh 

First idx:  25 ; Last idx:  24812 

Group  50  has  24646  number of rows in df_ts_fresh 

First idx:  167 ; Last idx:  24812 

Group  68  has  24790  number of rows in df_ts_fresh 

First idx:  23 ; Last idx:  24

In [7]:
first_idx = df.loc[df['group'] == 13, 'time_idx'].iloc[0]
total_time_idx = df[df.group == 13].shape[0]
reduced_df_time_idx = [*range(first_idx, int(dataset_size*total_time_idx)+1, 1)]
df = df[df['time_idx'].isin(reduced_df_time_idx)]

In [8]:
numeric_cols = list(df.columns)
numeric_cols = [e for e in numeric_cols if e not in ('time_idx', 'datetime', 'group', 'target', 'weather_description', 'weather_main')]
print(numeric_cols)

['inflow_lag_0', 'inflow_lag_1', 'inflow_lag_2', 'inflow_lag_3', 'outflow_lag_0', 'outflow_lag_1', 'outflow_lag_2', 'outflow_lag_3', 'inflow', 'inflow_diff1_per1', 'inflow_diff1_per2', 'inflow_diff2_per1', 'inflow_lag_0__fft_coefficient__attr_"real"__coeff_0', 'inflow_lag_0__fft_coefficient__attr_"real"__coeff_1', 'inflow_lag_0__skewness', 'inflow_lag_0__kurtosis', 'inflow_lag_0__count_above_mean', 'inflow_lag_0__count_below_mean', 'inflow_lag_0__autocorrelation__lag_1', 'inflow_lag_0__autocorrelation__lag_2', 'inflow_lag_0__autocorrelation__lag_3', 'inflow_lag_0__partial_autocorrelation__lag_1', 'inflow_lag_0__partial_autocorrelation__lag_2', 'inflow_lag_0__partial_autocorrelation__lag_3', 'inflow_lag_1__fft_coefficient__attr_"real"__coeff_0', 'inflow_lag_1__fft_coefficient__attr_"real"__coeff_1', 'inflow_lag_1__skewness', 'inflow_lag_1__kurtosis', 'inflow_lag_1__count_above_mean', 'inflow_lag_1__count_below_mean', 'inflow_lag_1__autocorrelation__lag_1', 'inflow_lag_1__autocorrelation

In [9]:
#################### add date information ts ####################

# create datetime features from datetime column in raw_feat_df
df['_hour_of_day'] = df["datetime"].dt.hour.astype(str)
df['_day_of_week'] = df["datetime"].dt.dayofweek.astype(str)
df['_day_of_month'] = df["datetime"].dt.day.astype(str)
df['_day_of_year'] = df["datetime"].dt.dayofyear.astype(str)
df['_week_of_year'] = df["datetime"].dt.weekofyear.astype(str)
df['_month_of_year'] = df["datetime"].dt.month.astype(str)
df['_year'] = df["datetime"].dt.year.astype(str)
#################### add date information ts ####################

hour_of_day_nunique = df['_hour_of_day'].nunique()
day_of_week_nunique = df['_day_of_week'].nunique()
day_of_month_nunique = df['_day_of_month'].nunique()
day_of_year_nunique = df['_day_of_year'].nunique()
week_of_year_nunique = df['_week_of_year'].nunique()
month_of_year_nunique = df['_month_of_year'].nunique()
year_nunique = df['_year'].nunique()

In [10]:
df

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,snow_1h,weather_description,weather_main,_hour_of_day,_day_of_week,_day_of_month,_day_of_year,_week_of_year,_month_of_year,_year
0,167,2021-01-07 23:00:00,4,0.0,10.0,8.0,1.0,1.0,5.0,3.0,...,0.00,16,0,23,3,7,7,1,1,2021
1,168,2021-01-08 00:00:00,4,1.0,4.0,13.0,0.0,2.0,0.0,1.0,...,0.00,16,0,0,4,8,8,1,1,2021
2,169,2021-01-08 01:00:00,4,2.0,1.0,9.0,0.0,0.0,1.0,2.0,...,0.00,16,0,1,4,8,8,1,1,2021
3,170,2021-01-08 02:00:00,4,0.0,1.0,5.0,0.0,1.0,2.0,3.0,...,0.00,16,0,2,4,8,8,1,1,2021
4,171,2021-01-08 03:00:00,4,0.0,3.0,2.0,1.0,1.0,0.0,0.0,...,0.00,16,0,3,4,8,8,1,1,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1607618,1235,2021-02-21 11:00:00,263,73.0,57.0,88.0,90.0,83.0,72.0,111.0,...,0.00,16,0,11,6,21,52,7,2,2021
1607619,1236,2021-02-21 12:00:00,263,102.0,60.0,96.0,121.0,81.0,73.0,113.0,...,0.00,16,0,12,6,21,52,7,2,2021
1607620,1237,2021-02-21 13:00:00,263,89.0,83.0,104.0,103.0,72.0,102.0,103.0,...,0.00,16,0,13,6,21,52,7,2,2021
1607621,1238,2021-02-21 14:00:00,263,72.0,72.0,114.0,100.0,82.0,89.0,98.0,...,0.00,16,0,14,6,21,52,7,2,2021


In [11]:
weather_description_nunique = df['weather_description'].nunique()
print("weather_description unique: ", weather_description_nunique)

weather_description unique:  17


In [12]:
weather_main_nunique = df['weather_main'].nunique()
print("weather_main unique: ", weather_main_nunique)

weather_main unique:  9


In [13]:
print(hour_of_day_nunique)
print(day_of_week_nunique)
print(day_of_month_nunique)
print(day_of_year_nunique)
print(week_of_year_nunique)
print(month_of_year_nunique)
print(year_nunique)

24
7
31
52
8
2
1


In [14]:
unique_values_group_no_ts_fresh = list(df['group'].unique())
print(len(unique_values_group_no_ts_fresh), "\n")
print(unique_values_group_no_ts_fresh)

66 

[4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 107, 113, 114, 116, 120, 125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153, 158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]


In [15]:
zones_time_idx_no_ts_fresh = {}
for i in unique_values_group_no_ts_fresh:
    df1 = df[df['group'] == i]
    time_idx_list = df1['time_idx'].tolist()
    first_idx = time_idx_list[0]
    last_idx = time_idx_list[-1]
    zones_time_idx_no_ts_fresh[i] = [first_idx, last_idx]
    print("Group ", i, " has ", df1.shape[0], " number of rows in df_ts_fresh \n")
    print("First idx: ", first_idx, "; Last idx: ", last_idx, "\n")

Group  4  has  1073  number of rows in df_ts_fresh 

First idx:  167 ; Last idx:  1239 

Group  12  has  1216  number of rows in df_ts_fresh 

First idx:  24 ; Last idx:  1239 

Group  13  has  1217  number of rows in df_ts_fresh 

First idx:  23 ; Last idx:  1239 

Group  24  has  1072  number of rows in df_ts_fresh 

First idx:  168 ; Last idx:  1239 

Group  41  has  1072  number of rows in df_ts_fresh 

First idx:  168 ; Last idx:  1239 

Group  42  has  1072  number of rows in df_ts_fresh 

First idx:  168 ; Last idx:  1239 

Group  43  has  1215  number of rows in df_ts_fresh 

First idx:  25 ; Last idx:  1239 

Group  45  has  1073  number of rows in df_ts_fresh 

First idx:  167 ; Last idx:  1239 

Group  48  has  1215  number of rows in df_ts_fresh 

First idx:  25 ; Last idx:  1239 

Group  50  has  1073  number of rows in df_ts_fresh 

First idx:  167 ; Last idx:  1239 

Group  68  has  1217  number of rows in df_ts_fresh 

First idx:  23 ; Last idx:  1239 

Group  74  has  

In [16]:
encoder_len = 8 # looks back 8 hours in the past for prediction
pred_len = 1 # 1, 6, 12 # how many timeseteps you want to predict in the future (forecasting), 1 means present (nowcasting)
#cov_lag_len= pred_len

# not using covariates yet, so put as 0
cov_lag_len= 0 #we can use forecasted values, even for inflow

####### train val test split of tsfresh dataframe #########

### you want a the idx for each train, validation and test

### train = 0.7 of each zone (eg. time idx 167 to roughly 20000) eg train = 0.7 * (24812-167)
### validation = 0.2 of each zone (eg. time idx 20000 to 22000)
### test = 0.1 of each zone (eg. time idx 22000 to 24000)

Target = 'target'

total_time_idx = df[df.group == 13].shape[0]

first_idx = df.loc[df['group'] == 13, 'time_idx'].iloc[0]

tr_stop_idx = int(0.7*total_time_idx)
val_start_idx = tr_stop_idx - encoder_len
val_stop_idx = total_time_idx-1
# val_stop_idx = val_start_idx + int(0.2*total_time_idx) # use (tr_stop_idx - encoder_len) as encoder from train set for validatoin
# tes_start_idx = val_stop_idx - encoder_len
# tes_stop_idx = total_time_idx-1

print(tr_stop_idx)
print(val_start_idx)
print(val_stop_idx)
# print(tes_start_idx)
# print(tes_stop_idx)

851
843
1216


In [17]:
train_time_idx = [*range(first_idx, first_idx+tr_stop_idx+1, 1)]
print(train_time_idx[0], train_time_idx[-1])
val_time_idx = [*range(first_idx+val_start_idx, first_idx+val_stop_idx+1, 1)]
print(val_time_idx[0], val_time_idx[-1])
# test_time_idx = [*range(first_idx+tes_start_idx, first_idx+tes_stop_idx+1, 1)]
# print(test_time_idx[0], test_time_idx[-1])

23 874
866 1239


In [18]:
# unique_zones = list(df['group'].unique())

In [19]:
"""
set inputs here
(hyperparameters grid search)

"""
######### Network Architecture ###################
# p = 10 # patience no. of epochs

Loss=NegativeBinomialDistributionLoss() # because we assume that the demand follows a negative bionmial distribution

######### Network Architecture ###################


######### Training Routine ###################
fdv_steps = 10 # fast_dev_run -> small set of training to see if training is happening correctly (one of the params in deepar model)
######### Training Routine ###################


# this is for naive forecast, getting past week's demand and predicting purely based on that
############## Inputs for 2) Persistance model ( seasonal naive forecast ) #######################
season_len = 168 # length of season ()
num_past_seas = 6 # number of past seasons to use in averaging (look at 6 week's back, getting average of these 6 values)
#seas_pred_strt_idx = 2035 # seasonal naive forecast start index, in hours use the df dataframe
############## Inputs for 2) Persistance model ( seasonal naive forecast ) #######################


In [20]:
"""
Full Training Routine
with hyperparmeter grid search

Load data into TimeSeriesDataSet object

for fast development run
uncomment fast_dev_run = fdv_steps

"""

# early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=p, verbose=False, mode="min")
lr_logger = LearningRateMonitor()

# param_comb_cnt=-1
#for neu,lay,bat,lr,enc_len,pred_len,drop,cov_pair,num_ep in product(*[x for x in hparams_grid.values()]):


file_path = "trial_parameters.json"
# Check if the file exists before attempting to delete it
if os.path.exists(file_path):
    # Delete the file
    os.remove(file_path)

import json
def save_trial_parameters(study, trial):
    trial_params = {'trial_num':trial.number ,'trial_params' :trial.params}
    with open("trial_parameters.json", "a") as f:
        json.dump(trial_params, f)
        f.write('\n')


def objective(trial): #optuna objective function to find best hyperparameters
# minimize the average validation loss

  # optuna tries to find best combination given this hyperparameter searchspace

  # first start broad
  neu = trial.suggest_int(name="neu",low=10,high=1000,step=100,log=False) # number of neurons in lstm cell
  lay = trial.suggest_int(name="lay",low=1,high=10,step=2,log=False) # number of lstm layers
  bat = trial.suggest_int(name="bat",low=32,high=1024,step=32,log=False) # batch size
  lr = trial.suggest_float(name="lr",low=0.00001,high=0.1,log=True) #learning rate, log does not have step
  drop = trial.suggest_float(name="dropout",low=0,high=0.8,step=0.2,log=False) # dont use some of the cells to distribute learning across the cells

  num_cols_list = numeric_cols
  cat_dict = {"_hour_of_day": NaNLabelEncoder(add_nan=True).fit(df._hour_of_day),
          "_day_of_week": NaNLabelEncoder(add_nan=True).fit(df._day_of_week), "_day_of_month" : NaNLabelEncoder(add_nan=True).fit(df._day_of_month), "_day_of_year" : NaNLabelEncoder(add_nan=True).fit(df._day_of_year),
              "_week_of_year": NaNLabelEncoder(add_nan=True).fit(df._week_of_year), "_month_of_year": NaNLabelEncoder(add_nan=True).fit(df._month_of_year) ,"_year": NaNLabelEncoder(add_nan=True).fit(df._year), "weather_description": NaNLabelEncoder(add_nan=True).fit(df.weather_description), "weather_main": NaNLabelEncoder(add_nan=True).fit(df.weather_main)}
  cat_list = ["_hour_of_day","_day_of_week","_day_of_month","_day_of_year","_week_of_year","_month_of_year","_year", "weather_description", "weather_main"]
  embed_size_dict = {'_hour_of_day':(hour_of_day_nunique+1,5),'_day_of_week':(day_of_week_nunique+1,5),'_day_of_month':(day_of_month_nunique+1,5), '_day_of_year':(day_of_year_nunique+1,8), '_week_of_year':(week_of_year_nunique+1,3), '_month_of_year':(month_of_year_nunique+1,3),'_year':(year_nunique+1,1), 'weather_description':(weather_description_nunique+1,5), 'weather_main':(weather_main_nunique+1,5)}
  # ^ converting hour into a 5 dimensional vector, etc etc, day of year = number of unique values in this column + 1, same for week of year, month of year and year

  train_dataset = TimeSeriesDataSet(
      df[(df['time_idx'].isin(train_time_idx))],
      time_idx="time_idx",
      target=Target,
      categorical_encoders=cat_dict,
      group_ids=["group"],
      min_encoder_length=encoder_len,
      max_encoder_length=encoder_len,
      min_prediction_length=pred_len,
      max_prediction_length=pred_len,
      time_varying_unknown_reals=[Target],
      time_varying_known_reals=num_cols_list,
      time_varying_known_categoricals=cat_list,
      add_relative_time_idx=False,
      randomize_length=False,
      scalers={},
      target_normalizer=TorchNormalizer(method="identity",center=False,transformation=None)
    )

  val_dataset = TimeSeriesDataSet.from_dataset(train_dataset,df[(df['time_idx'].isin(val_time_idx))], stop_randomization=True, predict=False)
  # test_dataset = TimeSeriesDataSet.from_dataset(train_dataset,df[df['time_idx'].isin(test_time_idx)], stop_randomization=True)

  train_dataloader = train_dataset.to_dataloader(train=True, batch_size=bat)
  val_dataloader = val_dataset.to_dataloader(train=False, batch_size=bat)
  # test_dataloader = test_dataset.to_dataloader(train=False, batch_size=bat)

  
  # trial.set_user_attr("num_ep", num_ep)

  early_stop_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=5,
    verbose=True,
    mode='min'
  )

  
  """
  Machine Learning predictions START
  1) DeepAR

  """
  trainer = pl.Trainer(
      max_epochs=num_epochs,
      accelerator='gpu',
      devices=[0],
      gradient_clip_val=0.1,
      limit_train_batches=1.0,
      limit_val_batches=1.0,
      # fast_dev_run=fdv_steps,
      logger=True,
      callbacks=[lr_logger, early_stop_callback],    #, early_stop_callback]
      default_root_dir=logs_directory
  )


  #print(f"training routing:\n \n {trainer}")
  deepar = DeepAR.from_dataset(
      train_dataset,
      learning_rate=lr,
      hidden_size=neu,
      rnn_layers=lay,
      dropout=drop,
      loss=Loss,
      log_interval=20,
      log_val_interval=6,
      log_gradient_flow=False,
      embedding_sizes=embed_size_dict
      # reduce_on_plateau_patience=3,
  )

  #print(f"Number of parameters in network: {deepar.size()/1e3:.1f}k")
  # print(f"Model :\n \n {deepar}")
  torch.set_num_threads(10)
  trainer.fit(
      deepar,
      train_dataloaders=train_dataloader,
      val_dataloaders=val_dataloader,
  )

  val_wmape = trainer.callback_metrics['val_WMAPE'].item()
  # val_wmape = trainer.callback_metrics['val_WMAPE'].item()
  trial.report(val_wmape, num_epochs)
  # trial.report(val_wmape, num_ep)

  # Handle pruning based on the intermediate value.
  if trial.should_prune():
      raise optuna.exceptions.TrialPruned()

  return val_wmape #optuna is trying to minimise this <---


######### optuna results #####################
if __name__ == "__main__":

  study = optuna.create_study(direction="minimize")
  study.optimize(objective, n_trials=num_trials,callbacks=[save_trial_parameters]) # can specify the number of trials to a number less than 960 for example, eg can set n_trials = 100

  pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
  complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

  print("Study statistics: ")
  print("  Number of finished trials: ", len(study.trials))
  print("  Number of pruned trials: ", len(pruned_trials))
  print("  Number of complete trials: ", len(complete_trials))

  print("Best trial:")
  trial = study.best_trial
  # best_num_ep = study.trials[trial.number].user_attrs['num_ep']

  print("  Value: ", trial.value)

  print("  Params: ")
  for key, value in trial.params.items():
      print("    {}: {}".format(key, value))
########## optuna results #####################

[I 2024-04-02 18:12:59,089] A new study created in memory with name: no-name-adcbacca-46cf-4ac4-aa1e-948f440e95a8
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
Missing logger folder: /home/optimusprime/Desktop/Timothy_EXP_4/Round_0_logs/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                             | Params
----------------------------------------------------------------------------
0 | loss                   | NegativeBinomialDistributionLoss | 0     
1 | logging_metrics        | ModuleList                       | 0     
2 | embeddings             | MultiEmbedding                   | 927   
3 | rnn                    | LSTM                     

Epoch 0: 100%|██████████| 2341/2341 [00:57<00:00, 40.88it/s, loss=3.97, v_num=0, train_loss_step=3.880, val_loss=4.250]

Metric val_loss improved. New best score: 4.245


Epoch 1: 100%|██████████| 2341/2341 [00:58<00:00, 40.11it/s, loss=4.06, v_num=0, train_loss_step=4.350, val_loss=4.110, train_loss_epoch=7.880]

Metric val_loss improved by 0.133 >= min_delta = 0. New best score: 4.113


...

Epoch 22: 100%|██████████| 81/81 [00:28<00:00,  2.89it/s, loss=2.33, v_num=29, train_loss_step=2.330, val_loss=2.620, train_loss_epoch=2.340]

Monitored metric val_loss did not improve in the last 5 records. Best score: 2.585. Signaling Trainer to stop.


Epoch 22: 100%|██████████| 81/81 [00:28<00:00,  2.86it/s, loss=2.33, v_num=29, train_loss_step=2.330, val_loss=2.620, train_loss_epoch=2.320]

[I 2024-04-02 23:40:49,509] Trial 29 finished with value: 31.0564022064209 and parameters: {'neu': 810, 'lay': 3, 'bat': 928, 'lr': 0.0030715803147346897, 'dropout': 0.2}. Best is trial 22 with value: 30.605316162109375.



Study statistics: 
  Number of finished trials:  30
  Number of pruned trials:  5
  Number of complete trials:  25
Best trial:
  Value:  30.605316162109375
  Params: 
    neu: 810
    lay: 1
    bat: 384
    lr: 0.0053935887501248515
    dropout: 0.4


In [21]:
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("  Best trial: ")
print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

# print("Best number of epochs: ", best_num_ep)

Study statistics: 
  Number of finished trials:  30
  Number of pruned trials:  5
  Number of complete trials:  25
  Best trial: 
  Value:  30.605316162109375
  Params: 
    neu: 810
    lay: 1
    bat: 384
    lr: 0.0053935887501248515
    dropout: 0.4


In [22]:
from IPython.display import display
fig = optuna.visualization.plot_parallel_coordinate(study)
display(fig)
fig = optuna.visualization.plot_optimization_history(study)
display(fig)
fig = optuna.visualization.plot_slice(study)
display(fig)
fig = optuna.visualization.plot_param_importances(study)
display(fig)

In [23]:
print("  Best trial: ", trial)

  Best trial:  FrozenTrial(number=22, state=TrialState.COMPLETE, values=[30.605316162109375], datetime_start=datetime.datetime(2024, 4, 2, 22, 37, 3, 302438), datetime_complete=datetime.datetime(2024, 4, 2, 22, 43, 0, 246051), params={'neu': 810, 'lay': 1, 'bat': 384, 'lr': 0.0053935887501248515, 'dropout': 0.4}, user_attrs={}, system_attrs={}, intermediate_values={30: 30.605316162109375}, distributions={'neu': IntDistribution(high=910, log=False, low=10, step=100), 'lay': IntDistribution(high=9, log=False, low=1, step=2), 'bat': IntDistribution(high=1024, log=False, low=32, step=32), 'lr': FloatDistribution(high=0.1, log=True, low=1e-05, step=None), 'dropout': FloatDistribution(high=0.8, log=False, low=0.0, step=0.2)}, trial_id=22, value=None)
